In [1]:
using Pkg
Pkg.activate(@__DIR__)

"/home/twan/.julia/dev/CentroidalTrajOpt/notebooks/Project.toml"

To do:

* Environment data should have its own axis, not shared with contacts
* Integer variables for contact pairs
* Make a type that stores the problem and all the variables
* Functions that transform coefficient arrays into e.g. piecewise Bezier curves
* Try with a final CoM position condition
* Initial condition modification

For output, want all of the quantities in the loop below as (vectors of) piecewise Bezier curves, and as a function of $t$, not $s$. Need to do postprocessing after `optimize!` call for this; `JuMP.value` won't cut it. This is also why it doesn't make sense to use `Piecewise` when setting up the constraints: there isn't an absolute $t$ variable that's shared between the pieces, just the $s_i$s for each individual piece.

In [2]:
using CentroidalTrajOpt
using SCIP

┌ Info: Recompiling stale cache file /home/twan/.julia/compiled/v1.1/CentroidalTrajOpt/LBRaS.ji for CentroidalTrajOpt [e21b34eb-f2c9-558f-afd6-4aa30e1b9e96]
└ @ Base loading.jl:1184


In [3]:
# Initial conditions
c0 = SVector(0.05, 0.1, 1.0)
ċ0 = SVector(0.25, 0.15, 0.1)

3-element StaticArrays.SArray{Tuple{3},Float64,1,3}:
 0.25
 0.15
 0.1 

In [7]:
problem = CentroidalTrajectoryProblem(with_optimizer(SCIP.Optimizer, limits_gap=CentroidalTrajOpt.Δt_tolerance), c0, ċ0);

In [8]:
solve!(problem)

presolving:
(round 1, fast)       106 del vars, 182 del conss, 0 add conss, 152 chg bounds, 0 chg sides, 0 chg coeffs, 8 upgd conss, 0 impls, 0 clqs
(round 2, fast)       106 del vars, 182 del conss, 0 add conss, 162 chg bounds, 0 chg sides, 0 chg coeffs, 8 upgd conss, 0 impls, 0 clqs
(round 3, fast)       106 del vars, 182 del conss, 0 add conss, 186 chg bounds, 0 chg sides, 0 chg coeffs, 8 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 106 deleted vars, 182 deleted constraints, 0 added constraints, 186 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 70 variables (0 bin, 0 int, 0 impl, 70 cont) and 62 constraints
      8 constraints of type <soc>
     12 constraints of type <linear>
     42 constraints of type <quadratic>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n| mem |mdpt |frac |vars |cons |cols |

In [ ]:
# TODO: update.
using Test
cval = value.(c)
c′val = value.(c′)
c′′val = value.(c′′)
ftotval = value.(sum(fs))
τval = value.(τ)
fsval = map(f -> value.(f), fs)
f̄sval = map(f̄ -> value.(f̄), f̄s)
τnsval = value.(τns)
Tval = T isa Number ? T : value(T) # TODO
@test map(x -> x(0), cval) ≈ c0 atol=1e-12
@test map(x -> x(0), c′val) ≈ ċ0 * Tval atol=1e-12
@test map(x -> x(1), c′′val) ≈ zeros(3) atol=1e-12
@test map(x -> x(1), c′′val) ≈ zeros(3) atol=1e-12
for poly in τval
   @test SVector(poly.coeffs) ≈ zeros(length(poly.coeffs)) atol=1e-6
end
for (fval, f̄val, τnval, transform, n, μ, μrot) in zip(fsval, f̄sval, τnsval, transforms, ns, μs, μrots)
    for i in 1 : C_NUM_COEFFS - 2
        f̄_control_point = map(x -> x.points[i], f̄val)
        @test norm(f̄_control_point[1 : 2]) <= μ * f̄_control_point[3] + 1e-6
    end
    for s in range(0, stop=1., length=100)
        f̄_s = map(x -> x(s), f̄val)
        @test norm(f̄_s[1 : 2]) <= μ * f̄_s[3] + 1e-6

        f_s = map(x -> x(s), fval)
        fn_s = n ⋅ f_s
        @test fn_s >= 0
        @test norm(f_s - n * fn_s) <= μ * fn_s + 1e-6
        
        τn_s = τnval(s)
        @test -μrot * fn_s <= τn_s
        @test τn_s <= μrot * fn_s
    end
end